In [38]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler
import numpy as np

In [39]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\heart.csv')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [41]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


# Preprocessing

In [42]:
numeric = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']
categorical = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall']

In [43]:
df_XGB = df.copy()
df_RF = df.copy()
df_SVM = df.copy()

In [44]:
df_SVM['sex'] =  df['sex'].astype('category')
df_SVM['cp'] =  df['cp'].astype('category')
df_SVM['fbs'] =  df['fbs'].astype('category')
df_SVM['restecg'] =  df['restecg'].astype('category')
df_SVM['exng'] =  df['exng'].astype('category')
df_SVM['slp'] =  df['slp'].astype('category')
df_SVM['caa'] =  df['caa'].astype('category')
df_SVM['thall'] =  df['thall'].astype('category')

In [45]:
df_SVM = pd.get_dummies(df)
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [46]:
df.shape

(303, 14)

# XGBoost

In [47]:
features = df_XGB.drop('output', axis=1)
targets = df_XGB['output']
train_X, test_X, train_y, test_y = train_test_split(features, targets, random_state=42)

XGBModel = XGBClassifier()

XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8147826086956522, 0.03668394315255914)

In [48]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8157894736842105

# XGB TUNING

In [49]:
XGBParam1 = {
    'gamma':[0.0,0.1,0.2],
    'learning_rate':[0.02, 0.025, 0.03],
    'max_depth': [2,4,6],
    'min_child_weight': [2,4,6],
    'n_estimators':[350, 375],
    'subsample':[0.6,0.7,0.8]
}

XGB_Grid1 = GridSearchCV(XGBClassifier(verbosity=0), XGBParam1)
XGB_Grid1.fit(train_X, train_y)
print(XGB_Grid1.best_params_, XGB_Grid1.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

{'gamma': 0.0, 'learning_rate': 0.025, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 350, 'subsample': 0.6} 0.8590338164251208


In [ ]:
XGBParam2 = {
    'learning_rate':[0.025,0.05,0.075,0.1],
    'max_depth': [3,4,5],
    'min_child_weight': [3,4,5],
    'n_estimators':[325,350],
    'subsample':[0.4,0.5,0.6]
}

XGB_Grid2 = GridSearchCV(XGBClassifier(verbosity=0,gamma=0.0), XGBParam2)
XGB_Grid2.fit(train_X, train_y)
print(XGB_Grid2.best_params_, XGB_Grid2.best_score_)

In [ ]:
XGBParam3 = {
    'max_depth': [1,2,3]
}

XGB_Grid3 = GridSearchCV(XGBClassifier(verbosity=0, gamma=0.0, n_estimators=350, min_child_weight=4, subsample=0.6, learning_rate=0.025), XGBParam3)
XGB_Grid3.fit(train_X, train_y)
print(XGB_Grid3.best_params_, XGB_Grid3.best_score_)

In [ ]:
XGBModel = XGBClassifier(verbosity=0,
                         gamma=0.0,
                         n_estimators=350,
                         min_child_weight=4,
                         subsample=0.6,
                         learning_rate=0.025,
                         max_depth=3)
XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

In [ ]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

# RandomForestClassifier

In [ ]:
features = df_RF.drop('output', axis=1)
targets = df_RF['output']
train_X, test_X, train_y, test_y = train_test_split(features, targets, random_state=42)

RFModel = RandomForestClassifier()

RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

In [ ]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

# RF TUNING

In [ ]:
RFParam1 = {"n_estimators":[350, 375],
            "max_depth":[2,4,6],
            'max_features':[5,10,15,20,31],
            "max_samples": [.1,.2,.3]
        }

RF_Grid1 = GridSearchCV(RandomForestClassifier(), RFParam1)
RF_Grid1.fit(train_X, train_y)
print(RF_Grid1.best_params_, RF_Grid1.best_score_)

In [ ]:
RFParam2 = {"max_depth":[3,4,5],
            'max_features':[12.5,15,17.5],
            "max_samples": [0.075,0.1,0.15]
            }

RF_Grid2 = GridSearchCV(RandomForestClassifier(n_estimators=375), RFParam2)
RF_Grid2.fit(train_X, train_y)
print(RF_Grid2.best_params_, RF_Grid2.best_score_)

In [ ]:
RFParam3 = {
            "max_samples": [0.05,0.065,0.075]
            }

RF_Grid3 = GridSearchCV(RandomForestClassifier(n_estimators=375, max_depth=4, max_features=15), RFParam3)
RF_Grid3.fit(train_X, train_y)
print(RF_Grid3.best_params_, RF_Grid3.best_score_)

In [ ]:
RFModel = RandomForestClassifier(n_estimators=375, max_depth=4, max_features=15, max_samples=0.065)

RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

In [ ]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

# Support Vector Machine

In [ ]:
features = df_SVM.drop('output', axis=1)
targets = df_SVM['output']
train_X, test_X, train_y, test_y = train_test_split(features, targets, random_state=42)

scaler = RobustScaler()
features[numeric] = scaler.fit_transform(features[numeric])

In [ ]:
SVMModel = SVC()

SVM_scores = cross_val_score(SVMModel, train_X, train_y)
(SVM_scores.mean(), SVM_scores.std())

In [ ]:
SVMModel.fit(train_X, train_y)
pred = SVMModel.predict(test_X)
accuracy_score(test_y, pred)

# SVM TUNING

In [ ]:
SVMParam = {"C":np.arange(1,10,1),
            'gamma':np.arange(0.0125,1, 0.0125)}
SVM_Grid = GridSearchCV(SVC(), SVMParam)
SVM_Grid.fit(train_X, train_y)
print(SVM_Grid.best_params_, SVM_Grid.best_score_)

In [ ]:
SVMModel = SVC(C=3, gamma=0.0375)

SVM_scores = cross_val_score(SVMModel, train_X, train_y)
(SVM_scores.mean(), SVM_scores.std())

In [ ]:
SVMModel.fit(train_X, train_y)
pred = SVMModel.predict(test_X)
accuracy_score(test_y, pred)